In [ ]:
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import math
from math import exp
from random import randint
import random
import numpy as np
from matplotlib import pyplot
x_train,y_train=load_svmlight_file('/home/chen/桌面/a9a.txt')#读取数据
x_test,y_test=load_svmlight_file('/home/chen/桌面/a9at.txt',n_features=123)
x_train=x_train.toarray()
x_train=np.hstack((x_train,np.ones([np.shape(x_train)[0],1])))
y_train=y_train.reshape(np.shape(y_train)[0],1)
x_test=x_test.toarray()
x_test=np.hstack((x_test,np.ones([np.shape(x_test)[0],1])))
n=np.shape(x_train)[0]
m=np.shape(x_train)[1]


def loss(X,Y,W):
    n,m=X.shape
    l=0
    for i in range(n):
        l+=math.log((1+exp(-Y[i]*W.T.dot(X[i]))),math.e)
    return l/n

def gradient(X,Y,W):
    n,m=X.shape
    g=0
    for i in range(n):
        g-=Y[i]*X[i]/(1+exp(Y[i]*W.T.dot(X[i])))
    return (g/n).reshape((g.shape[0],1))


"""NAG"""
W=np.zeros((m,1))
L_NAG=[]
eta=0.1
gamma=0.1
batch=100

v=0
for epoch in range(1500):
    random.seed()
    i=randint(0,n-1-batch)
    g=gradient(x_train[i:i+batch].reshape((batch,m)),y_train[i:i+batch].reshape((batch,1)),W-gamma*v)
    v=gamma*v+eta*g
    W=W-v
    l_test=loss(x_test,y_test,W)
    L_NAG.append(l_test)
print(L_NAG[-1])
print('finished training')

"""RMSProp"""
W=np.zeros((m,1))
L_RMSProp=[]
eta=0.01
gamma=0.9
epsilon=1e-3
batch=100

G=0
for epoch in range(1500):
    random.seed()
    i=randint(0,n-1-batch)
    g=gradient(x_train[i:i+batch].reshape((batch,m)),y_train[i:i+batch].reshape((batch,1)),W)
    G=gamma*G+(1-gamma)*(g*g)
    W=W-eta/np.sqrt(G+epsilon)*g
    l_test=loss(x_test,y_test,W)
    L_RMSProp.append(l_test)
print(L_RMSProp[-1])
print('finished training')

"""AdaDelta"""
W=np.zeros((m,1))
L_AdaDelta=[]
gamma=0.9
epsilon=1e-6
batch=100

G=0
dt=0
for epoch in range(1500):
    random.seed()
    i=randint(0,n-1-batch)
    g=gradient(x_train[i:i+batch].reshape((batch,m)),y_train[i:i+batch].reshape((batch,1)),W)
    G=gamma*G+(1-gamma)*g*g
    dw=-np.sqrt(dt+epsilon)/np.sqrt(G+epsilon)*g
    W=W+dw
    dt=gamma*dt+(1-gamma)*dw*dw
    l_test=loss(x_test,y_test,W)
    L_AdaDelta.append(l_test)
print(L_AdaDelta[-1])
print('finished training')


"""Adam"""
W=np.zeros((m,1))
L_Adam=[]
beta=0.9
gamma=0.999
eta=0.001
epsilon=1e-6
batch=100

M=0
for epoch in range(1500):
    i=randint(0,n-1-batch)
    g=gradient(x_train[i:i+batch].reshape((batch,m)),y_train[i:i+batch].reshape((batch,1)),W)
    M=beta*M+(1-beta)*g
    G=gamma*G+(1-gamma)*g*g
    alpha=eta*np.sqrt(1-math.pow(gamma,epoch))/(1-beta)
    W=W-alpha*M/np.sqrt(G+epsilon)
    l_test=loss(x_test,y_test,W)
    L_Adam.append(l_test)
print(L_Adam[-1])
print('finished training')


pyplot.ylabel("loss")
pyplot.yscale("log")
pyplot.plot(L_NAG,label='NAG')
pyplot.plot(L_RMSProp,label='MSProp')
pyplot.plot(L_AdaDelta,label='AdaDelta')
pyplot.plot(L_Adam,label='Adam')
pyplot.legend()
pyplot.show()